In [2]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from Backend import perf_calc as perf_calc
from Backend import rotation_calc as rotation_calc
from Backend import refusal as refusal
from database import database_handler
from fastapi.responses import JSONResponse
from Backend import handle_route as handle_route
from pydantic import BaseModel 
from Backend import wind_calc
import sqlite3

g:\CompSciProject
g:\CompSciProject
g:\CompSciProject
g:\CompSciProject


In [11]:
def search_airport(partial_name):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    words = partial_name.split()
    conditions = " AND ".join(["name LIKE ? COLLATE NOCASE" for _ in words])

    query = f"""
    SELECT name, icao,type
    FROM airports 
    WHERE {conditions} 
    ORDER BY 
        LENGTH(name), 
        INSTR(LOWER(name), LOWER(?)) 
    """

    params = tuple(f"%{word}%" for word in words) + (words[0],)

    cursor.execute(query, params)
    results = cursor.fetchall()
    connect_to_db.close()

    return results
search_airport("hello")

[('Othello Municipal Airport', 'KS70', 'small_airport')]

In [ ]:
import sqlite3
import json

db_path = "database/nav_data.db"
json_path = "C:/Users/Thomas/Downloads/Airport-Codes-main/Airport-Codes-main/airports.json"

with open(json_path, "r") as file:
    airports_data = json.load(file)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE airports ADD COLUMN name TEXT;")
except sqlite3.OperationalError:
    print("Column 'name' already exists.")

update_data = [(data.get("name", ""), icao) for icao, data in airports_data.items()]

cursor.executemany("UPDATE airports SET name = ? WHERE icao = ?", update_data)

conn.commit()
conn.close()

print("Database update completed successfully.")


Database update completed successfully.
